# 名詞・形容詞ペアの抽出によるレビュー分析

## 1. 環境構築
GiNZAのインストール

In [1]:
! pip install -U ginza ja_ginza

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 1.1 MB/s 
     |████████████████████████████████| 2.2 MB 60.5 MB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574782 sha256=2a49b7df4926953c4f40c83a8645bf9e4c17c22f552fbd3102d40dd5dfe783fd
  Stored in directory: /root/.cache/pip/wheels/ca/e3/ed/e78fecf6fd34349114d292242a16fc08d513fb32c2d9c5d786
Successfully built SudachiDict-core


## 2. GiNZAで形態素解析+係り受け分析

In [2]:
# GiNZAの日本語モデルと読み込む
import spacy
nlp = spacy.load('ja_ginza')

doc = nlp("温かい雰囲気が素敵。")
for token in doc:
    print(token.text, token.lemma_, token.pos_)  # 表層形, 原形, 品詞

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


温かい 温かい ADJ
雰囲気 雰囲気 NOUN
が が ADP
素敵 素敵 ADJ
。 。 PUNCT


### 日本語品詞情報を取得するために、spaCy.tokens.Tokenを拡張する

In [3]:
# 品詞情報を取得するためTokenを拡張する
from spacy.tokens import Token

def get_jp_pos(token, idx):
    # UniDicの品詞体系
    # https://hayashibe.jp/tr/mecab/dictionary/unidic/pos
    pos_list = token.tag_.split('-')
    if idx < len(pos_list):
        return pos_list[idx]
    return None

# Token.set_extension
# cf. https://spacy.io/api/token
Token.set_extension("jp_pos_main", getter=lambda x: get_jp_pos(x, 0))
Token.set_extension("jp_pos_sub", getter=lambda x: get_jp_pos(x, 1))

doc = nlp("温かい雰囲気が素敵。")
for token in doc:
    print(token.text, token._.jp_pos_main, token._.jp_pos_sub)

温かい 形容詞 一般
雰囲気 名詞 普通名詞
が 助詞 格助詞
素敵 形状詞 一般
。 補助記号 句点


### 係り受け解析の結果例

In [4]:
from spacy import displacy

doc = nlp("温かい雰囲気が素敵。")
displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

for token in doc:
    for child_token in token.children:
        print('{} --> {} : {}'.format(token.text, child_token.text, child_token.dep_))

雰囲気 --> 温かい : acl
雰囲気 --> が : case
素敵 --> 雰囲気 : nsubj
素敵 --> 。 : punct


## 3. 複数の形態素からなる形容詞を抜き出す

In [5]:
def compound_adjective(token):
    """複合語の形容詞を返す. 該当しない場合はNone."""
    # 次のtokenを取得
    next_token = token.doc[token.i+1] if not token.is_sent_end else None
    prev_token = token.doc[token.i-1] if not token.is_sent_start else None

    if next_token is not None:
        # 形容動詞 （形状詞 + な） ... 静かな
        if (token._.jp_pos_main == '形状詞') and (next_token.text == 'な'):
            return token.text + 'な'
        # 名詞 + な  ... 元気な
        if (token._.jp_pos_main == '名詞') and (next_token.text == 'な'):
            return token.text + 'な'
        # 動詞 + 形容詞的 ... わかりやすい, 怒りっぽい
        if (token._.jp_pos_main == '動詞') and (next_token._.jp_pos_sub == '形容詞的'):
            return token.text + next_token.text

    if prev_token is not None:
        # 名詞 + ない ... 問題ない, 仕方ない
        if (prev_token._.jp_pos_main == '名詞') and (token._.jp_pos_main == '形容詞') and (token._.jp_pos_sub == '非自立可能'):
            return prev_token.text + token.text

    # 該当しない場合は None
    return None

texts = [
    'この本はわかりやすい。',
    '機能としては問題ない',
    '高価な腕時計をした男性。',
    '穏やかな人。',
    '先生は怒りっぽい。',
]

for text in texts:
    print('=' * 30)
    print(text)
    doc = nlp(text)
    for token in doc:
        ret = compound_adjective(token)
        if ret:
            print('- {} --> {}'.format(token.text, ret))

この本はわかりやすい。
- わかり --> わかりやすい
機能としては問題ない
- ない --> 問題ない
高価な腕時計をした男性。
- 高価 --> 高価な
穏やかな人。
- 穏やか --> 穏やかな
先生は怒りっぽい。
- 怒り --> 怒りっぽい


### 形容詞として抽出できないケース

In [6]:
doc = nlp("緊迫した雰囲気。")
for token in doc:
    print(token.text, token._.jp_pos_main, token._.jp_pos_sub)
    ret = compound_adjective(token)
    if ret:
        print('( {} --> {} )'.format(token.text, ret))

緊迫 名詞 普通名詞
し 動詞 非自立可能
た 助動詞 None
雰囲気 名詞 普通名詞
。 補助記号 句点


In [7]:
doc = nlp("落ち着いた雰囲気。")
for token in doc:
    print(token.text, token._.jp_pos_main, token._.jp_pos_sub)
    ret = compound_adjective(token)
    if ret:
        print('( {} --> {} )'.format(token.text, ret))

落ち着い 動詞 一般
た 助動詞 None
雰囲気 名詞 普通名詞
。 補助記号 句点


## 4. 名詞・形容詞のペアを抽出する
係り受け解析の結果（UD）が以下のケースを抽出対象とする
- 名詞 --> 形容詞：acl
- 形容詞 --> 名詞：nsubj

In [8]:
def is_adjective(token):
    """形容詞か否かと、形容詞の場合はその文字列を返す."""
    comp_adj = compound_adjective(token)
    if comp_adj:
        return True, comp_adj
    elif token._.jp_pos_main == '形容詞':
        return True, token.lemma_
    return False, None


def get_noun_adj_pairs(text):
    """名詞と形容詞のペアを返す."""
    noun_types = ('名詞')  #('名詞', '代名詞')
    pairs = []
    doc = nlp(text)
    for token in doc:
        par_is_adj, par_adj_text = is_adjective(token)
        for child_token in token.children:
            if par_is_adj and (child_token._.jp_pos_main in noun_types) and (child_token.dep_ == 'nsubj'):
                pairs.append([child_token.text, par_adj_text])
            elif (token._.jp_pos_main in noun_types):
                chi_is_adj, chi_adj_text = is_adjective(child_token)
                if chi_is_adj and (child_token.dep_ == 'acl'):
                    pairs.append([token.text, chi_adj_text])
    return pairs

text = 'スープが美味しい。麺は縮れ麺で、スープと相性がよい。値段はやや高いものの、総じて満足感は高い。'
get_noun_adj_pairs(text)

[['スープ', '美味しい'], ['相性', 'よい'], ['値段', '高い'], ['満足感', '高い']]

In [9]:
# 他の例文にも適用してみる
texts = [
    'この本はわかりやすい。',
    '夕ご飯に作った味噌汁の味が少し濃かった。',
    '先生は優しい。',
    'このホテルの朝食は美味しい。',
    '美味しいご飯に、温かい布団。',
    '美しい景色が素晴らしい。',
    '高価な腕時計をした男性。',
    '穏やかな人。',
    'その間違いは仕方ない。',
    '問題ない難易度。',
    # 誤り？の例
    '彼は怒りっぽい。',              # 名詞ではなく代名詞
    '彼女はいつも明るい。',          # 「いつも」が名詞？
    '先輩の説明に問題はなかった。',  # 「ない」は形容詞？
    '議員の発言はとんでもない。',    # 「とんでもない」を形容詞として取れない
]

for text in texts:
    print('=' * 30)
    print(text)
    print('-' * 30)
    for noun, adj in get_noun_adj_pairs(text):
        print('  ({}, {})'.format(noun, adj))

この本はわかりやすい。
------------------------------
  (本, わかりやすい)
夕ご飯に作った味噌汁の味が少し濃かった。
------------------------------
  (味, 濃い)
先生は優しい。
------------------------------
  (先生, 優しい)
このホテルの朝食は美味しい。
------------------------------
  (朝食, 美味しい)
美味しいご飯に、温かい布団。
------------------------------
  (ご飯, 美味しい)
  (布団, 温かい)
美しい景色が素晴らしい。
------------------------------
  (景色, 美しい)
  (景色, 素晴らしい)
高価な腕時計をした男性。
------------------------------
  (腕時計, 高価な)
穏やかな人。
------------------------------
  (人, 穏やかな)
その間違いは仕方ない。
------------------------------
  (間違い, 仕方ない)
問題ない難易度。
------------------------------
  (難易度, 問題ない)
彼は怒りっぽい。
------------------------------
彼女はいつも明るい。
------------------------------
  (いつも, 明るい)
先輩の説明に問題はなかった。
------------------------------
  (問題, ない)
議員の発言はとんでもない。
------------------------------
  (発言, ない)


## 5. 食べログレビューに適用してみる
横浜のフレンチレストラン「霧笛楼」のレビューに適用してみる
- タイトル10件
- レビュー本文1件

In [10]:
review_titles = [
    '優雅でエレガンスな西洋の雰囲気♡格調高い気品のあるクラシックな空間で「横濱フレンチ」♡',
    '元町のグランメゾン！素敵な洋館で優雅なランチ！',
    '横濱仏蘭西料理の伝統と革新',
    '『開港当時の古き良き横浜に思いを馳せて』',
    '想い出の霧笛楼',
    'お値段以上に凝ったお料理でしたが、過度な火入れは苦痛かも？',
    '霧笛（むてき）歓待',
    '開港地区には異国情緒の風が吹く',
    '霧笛楼40周年記念　冬の特別メニューをいただきました',
    '個室がオススメ 接客料理ともに◎'
]

noun_adj_pairs = []
for text in review_titles:
    noun_adj_pairs += get_noun_adj_pairs(text)

noun_adj_pairs

[['雰囲気', 'エレガンスな'],
 ['気品', '高い'],
 ['空間', 'クラシックな'],
 ['洋館', '素敵な'],
 ['ランチ', '優雅な'],
 ['横浜', '良し'],
 ['火入れ', '過度な']]

In [11]:
# https://tabelog.com/kanagawa/A1401/A140105/14001146/dtlrvwlst/B439819004/?use_type=0&smp=1
text = \
"""開港当時に人気だった港崎町遊郭の料亭『岩亀楼』をイメージした佇まいの〈仏蘭西料亭 横濱元町 霧笛楼〉は、横濱フレンチ”を愉しめる老舗レストラン。
チョコレートのお菓子も有名なので、よくお土産で頂いたり、贈ったりしています。
この日は、私がHSK6級最難関合格したお祝いです。(*'▽'*)
(中国語検定です)
レストラン霧笛楼は、外観からもう私の好みにドンピシャ♡
内装や調度品・器に至るまで、異国情緒溢れる古き良き横浜の世界観を再現すべくこだわりの空間は本当に素敵です。
異国情緒あふれるハイカラでオリエンタルな雰囲気に思わず引きこまれてしまうような…
お皿もどれも私の好みで…♡
実は、レトロ西洋建築がすごく好きなんです。
それこそ、明治、大正時代の建物がたまらなくて好き。
ジブリだと、風立ちぬが1番好きです。
あまりに美しくて切なくてな感じ。
建物も風景も人も。。
風真摯に仕事に向き合う姿勢と、純粋に人を愛することが両方同じくらい大切なことと描かれてるのがたまらなく好き。
ジブリで1番好きですね。
さて、開港後を感じさせるきれいな絵皿が異国情緒を感じさせてくれる演出。
食事はシェフの横浜らしい美味しさを味わっていただきたいとのことで、横濱フレンチ。
3つのコンセプトを守り続けています。
1つ、「和魂洋才」。優雅でエレガンスな西洋の雰囲気を感じていただきながら、気取らずに日本の心でフランス料理や空間を提供する。
2つ、「温故知新」。本場フレンチの技法に学び、素晴らしい和の出汁文化や先人の教えに真摯に向き合い、日本ならではの季節感を大切に口の中で渾然一体となる一皿を目指す。
3つ、「地産地消」。食材を活かすポイントである“新鮮さ”、とりわけ野菜は収穫から調理までのタイムラグを出来るだけなくし、地元の畑で採れる新鮮な食材を多く使用する。
見た目にも美しく、同じ食材でも味わったことない美味しさで、サービスも当然申し分なく、全てが素晴らしい体験。
所謂日本の伝統的なフレンチで美味しい。
フレンチに和の味わいを感じます。
クラシカルな雰囲気の中で、正統派でありながら進化し続けるフレンチを味わえる贅沢を感じられました。
接客は申し分なく対応も完璧でした。(*'▽'*)"""

noun_adj_pairs = get_noun_adj_pairs(text)
noun_adj_pairs

[['お菓子', '有名な'],
 ['世界観', '良し'],
 ['雰囲気', 'オリエンタルな'],
 ['建築', '好きな'],
 ['感じ', '切ない'],
 ['こと', '大切な'],
 ['こと', '大切な'],
 ['絵皿', 'きれいな'],
 ['雰囲気', 'エレガンスな'],
 ['和', '素晴らしい'],
 ['食材', '新鮮な'],
 ['美味しさ', 'ことない'],
 ['サービス', '申し分なく'],
 ['全て', '素晴らしい'],
 ['体験', '素晴らしい'],
 ['フレンチ', '伝統的な'],
 ['雰囲気', 'クラシカルな'],
 ['接客', '申し分なく']]

## 参考文献
1. [河野ら, "不特定分野の商品レビューを対象とした評価情報の自動認識" (2017)](https://cir.nii.ac.jp/crid/1050574047123697408)
2. [M3 Tech Blog - GiNZAと患者表現辞書を使って患者テキストの表記ゆれを吸収した意味構造検索を試した](https://www.m3tech.blog/entry/meaning-structure-search)
3. [金山ら, "日本語 Universal Dependencies の試案" (2015)](https://www.anlp.jp/proceedings/annual_meeting/2015/pdf_dir/E3-4.pdf)
4. [Universal Dependencies の日本語の情報ページ](https://udjapanese.github.io/docs/)